# Tutorial sobre CBRkit

CBRkit es una librería en Python para el desarrollo de sistemas CBR. Se centra principalmente en la etapa de recuperación y tiene un catálogo bastante amplio de medidas de similitud. 

Principales enlaces:

- Repositorio: <https://github.com/wi2trier/cbrkit/>
- Documentación: <https://wi2trier.github.io/cbrkit/cbrkit.html>
- Tests: <https://github.com/wi2trier/cbrkit/tree/main/tests> Sirven como ejemplo de código actualizado (hay un proyecto de demo pero está obsoleto)

## Instalación

Para instalar CBRkit usamos `pip install cbrkit`. Sin embargo, CBRkit tiene dependencias opcionales que hay que instalar aparte. Si quisiésemos instalar todo usaríamos `pip install cbrkit[all]` pero se recomienda instalar solo las necesarias (para esta demo solo hemos instalado las de nlp, tal y como aparece en el archivo pyproject.toml). Las dependencias que tiene son las siguiente:

- `all`: All optional dependencies
- `api`: REST API Server
- `cli`: Command Line Interface (CLI)
- `eval`: Evaluation tools for common metrics like `precision` and `recall`
- `graphs`: Graph libraries like `networkx` and `rustworkx`
- `llm`: Large Language Models (LLM) APIs like Ollama and OpenAI
- `nlp`: Standalone NLP tools `levenshtein`, `nltk`, `openai`, and `spacy`
- `timeseries`: Time series similarity measures like `dtw` and `smith_waterman`
- `transformers`: Advanced NLP tools based on `pytorch` and `transformers`

## Primeros pasos

### Cargar una base de casos

CBRkit tiene cargadores por defecto para distintos formatos de archivos (csv, json, toml, cml, yaml, y objetos Python en archivos con extensión .py ). Cualquiera de ellos se puede cargar usando `cbrkit.loaders.file` (que usará el cargador correspondiente). Al hacer esto, se creará una base de casos en la que para cada caso se le asignará como id un entero consecutivo comenzando en 0.

A la base de casos se puede acceder como a un diccionario, usando el id del caso. Internamente se usa [Polars](https://pola.rs/), que es una librería de dataframes similar a la de pandas.

In [1]:
import cbrkit
import os

# Cargamos la base de casos de 1000 coches usados que está en ./data
carsCasebase = cbrkit.loaders.file(os.path.join(os.path.abspath(""),"data", "cars-1k.csv"))
carsCasebase


/Users/guille/gitRepos/cbrkit-tutorial/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


polars(df=shape: (1_000, 11)
┌───────┬──────┬──────────────┬──────────────────┬───┬──────────────┬───────┬────────┬─────────────┐
│ price ┆ year ┆ manufacturer ┆ model            ┆ … ┆ transmission ┆ drive ┆ type   ┆ paint_color │
│ ---   ┆ ---  ┆ ---          ┆ ---              ┆   ┆ ---          ┆ ---   ┆ ---    ┆ ---         │
│ i64   ┆ i64  ┆ str          ┆ str              ┆   ┆ str          ┆ str   ┆ str    ┆ str         │
╞═══════╪══════╪══════════════╪══════════════════╪═══╪══════════════╪═══════╪════════╪═════════════╡
│ 30590 ┆ 2020 ┆ subaru       ┆ brz limited      ┆ … ┆ other        ┆ rwd   ┆ coupe  ┆ white       │
│       ┆      ┆              ┆ coupe 2d         ┆   ┆              ┆       ┆        ┆             │
│ 17500 ┆ 2013 ┆ ram          ┆ 2500             ┆ … ┆ automatic    ┆ 4wd   ┆ pickup ┆ white       │
│ 7800  ┆ 2010 ┆ honda        ┆ accord           ┆ … ┆ automatic    ┆ fwd   ┆ sedan  ┆ black       │
│ 37990 ┆ 2016 ┆ toyota       ┆ tundra double    ┆ … ┆ other  

### Definir funciones de similitud

La definición de la función de similitud se basa en la definición de una función de similitud global, así como las funciones de similitud locales para cada uno de los atributos que se quieran utilizar. Aunque tiene distintas formas de definir las funciones de similitud, vamos a comenzar con las más sencillas, que son las que se usan en casos del tipo atributo-valor (`cbrkit.sim.attribute_value`):

- La función de similitud global se define en el `cbrkit.sim.aggregator`. Con el parámetro `pooling` definimos qué función queremos utilizar [de entre un catálogo que tiene CBRkit](https://wi2trier.github.io/cbrkit/cbrkit/sim/pooling.html): mean, fmean, geametric_mean, median, max...
- Para cada uno de los atributos definimos la función de similitud local que queremos aplicar. De nuevo, CBRkit proporciona [un gran catálogo de funciones de similitud](https://wi2trier.github.io/cbrkit/cbrkit/sim.html) para distintos tipos de datos: números, cadenas, colecciones, grafos, embeddings (vectores), taxonomías...




In [2]:
# Definimos una función de similitud para la base de casos de coches
simFunction = cbrkit.sim.attribute_value(
    attributes={
        # Para el año usamos una función de similitud lineal
        # (entre un mínimo y un máximo, de acuerdo a los datos de la base de casos)
        "year": cbrkit.sim.numbers.linear_interval(1950, 2021),

        # Para el manufacturer, utilizamos la comparación de cadenas usando la distancia de Levenshtein
        "manufacturer": cbrkit.sim.strings.levenshtein(),
    },
    # Como medida de similitud global utilizamos la media 
    aggregator=cbrkit.sim.aggregator(pooling="mean"),
)

### Recuperación de casos

Para poder ejecutar la fase CBR de recuperación necesitamos:
- Configurar una estrategia de recuperación o `retriever`
- Definir una consulta: un diccionario con los atributos y valores que necesitemos. Una consulta puede ser un caso de la base de casos.
- Ejecutar la consulta usando `cbrkit.retrieval.apply_query`.

El resultado de la ejecución se almacena en un objeto de tipo [Result](https://github.com/wi2trier/cbrkit/blob/main/src/cbrkit/model/result.py). Los atributos más importantes son los siguientes:

- `result.similarities`: Contiene el cálculo de similitud de la consulta con el resto de casos en la base de casos. Produce información tanto de la similitud global como de las similitudes locales. 
- `result.ranking`: una secuencia con los ids de los casos ordenados por similitud decreciente.
- `result.duration`: tiempo que ha tardado en ejecutarse la consulta. 


In [3]:
# Creamos el retriever
retriever = cbrkit.retrieval.build(simFunction)
# Creamos una consulta
query = {"year": 2000, "manufacturer": "mazda"}
# Ejecutamos la consulta
result = cbrkit.retrieval.apply_query(carsCasebase, query, retriever)

# Revisamos los valores de similitud
result.similarities

{0: AttributeValueSim(value=0.45006402048655564, attributes={'year': 0.7183098591549295, 'manufacturer': 0.18181818181818177}),
 1: AttributeValueSim(value=0.5334507042253521, attributes={'year': 0.8169014084507042, 'manufacturer': 0.25}),
 2: AttributeValueSim(value=0.6295774647887324, attributes={'year': 0.8591549295774648, 'manufacturer': 0.4}),
 3: AttributeValueSim(value=0.4782330345710627, attributes={'year': 0.7746478873239436, 'manufacturer': 0.18181818181818177}),
 4: AttributeValueSim(value=0.5204865556978233, attributes={'year': 0.8591549295774648, 'manufacturer': 0.18181818181818177}),
 5: AttributeValueSim(value=0.5266040688575899, attributes={'year': 0.8309859154929577, 'manufacturer': 0.2222222222222222}),
 6: AttributeValueSim(value=0.5123239436619718, attributes={'year': 0.7746478873239436, 'manufacturer': 0.25}),
 7: AttributeValueSim(value=0.6577464788732394, attributes={'year': 0.9154929577464789, 'manufacturer': 0.4}),
 8: AttributeValueSim(value=0.4154929577464789

In [4]:
# Ids de los casos, en orden decreciente de similitud
result.ranking

(248,
 138,
 638,
 984,
 328,
 97,
 249,
 188,
 474,
 794,
 460,
 433,
 954,
 365,
 765,
 158,
 897,
 609,
 546,
 854,
 630,
 498,
 387,
 266,
 475,
 604,
 830,
 778,
 369,
 285,
 321,
 7,
 868,
 242,
 355,
 96,
 590,
 607,
 747,
 105,
 75,
 489,
 600,
 930,
 939,
 763,
 759,
 25,
 364,
 394,
 523,
 2,
 257,
 492,
 507,
 671,
 598,
 404,
 646,
 650,
 688,
 869,
 959,
 608,
 655,
 381,
 537,
 714,
 203,
 259,
 354,
 627,
 628,
 987,
 990,
 261,
 431,
 483,
 506,
 565,
 168,
 278,
 352,
 427,
 98,
 202,
 440,
 887,
 896,
 380,
 704,
 916,
 439,
 186,
 250,
 398,
 881,
 128,
 204,
 863,
 867,
 14,
 468,
 542,
 911,
 119,
 823,
 132,
 184,
 462,
 809,
 989,
 690,
 699,
 58,
 78,
 408,
 770,
 824,
 829,
 986,
 180,
 251,
 444,
 601,
 683,
 909,
 144,
 164,
 656,
 757,
 393,
 594,
 610,
 641,
 346,
 471,
 514,
 549,
 635,
 889,
 24,
 978,
 32,
 140,
 532,
 941,
 983,
 123,
 491,
 522,
 945,
 100,
 831,
 412,
 66,
 165,
 280,
 665,
 678,
 944,
 962,
 125,
 356,
 397,
 899,
 955,
 53,
 545,
 5

In [5]:
# Podemos ver la información de caso más similar a la consulta 
print(
    "Consulta: ",
    query,
    os.linesep,
    "Similitud: ",
    result.similarities[result.ranking[0]],
    os.linesep,
    "Caso: ",
    carsCasebase[result.ranking[0]]
)

Consulta:  {'year': 2000, 'manufacturer': 'mazda'} 
 Similitud:  AttributeValueSim(value=0.9859154929577465, attributes={'year': 0.971830985915493, 'manufacturer': 1.0}) 
 Caso:  {'price': 5990, 'year': 2002, 'manufacturer': 'mazda', 'model': 'tribute', 'condition': 'good', 'fuel': 'gas', 'title_status': 'clean', 'transmission': 'automatic', 'drive': '4wd', 'type': 'SUV', 'paint_color': 'grey'}


En lugar de utilizar directamente el retriever CBRkit proporciona una clase [`Dropout`](wi2trier.github.io/cbrkit/cbrkit/retrieval.html#dropout) que aplica ordenación y filtros sobre el retreiver, de modo que facilita el proceso de recuperación de casos similares.

In [6]:
# Usamos el dropout para quedarnos con los 10 casos más similares.
retriever = cbrkit.retrieval.dropout(
    cbrkit.retrieval.build(simFunction), # A retriever
    limit=10,
)

query = {"year": 2000, "manufacturer": "mazda"}
result = cbrkit.retrieval.apply_query(carsCasebase, query, retriever)
result.similarities

{248: AttributeValueSim(value=0.9859154929577465, attributes={'year': 0.971830985915493, 'manufacturer': 1.0}),
 138: AttributeValueSim(value=0.9788732394366197, attributes={'year': 0.9577464788732395, 'manufacturer': 1.0}),
 638: AttributeValueSim(value=0.9295774647887324, attributes={'year': 0.8591549295774648, 'manufacturer': 1.0}),
 984: AttributeValueSim(value=0.9225352112676056, attributes={'year': 0.8450704225352113, 'manufacturer': 1.0}),
 328: AttributeValueSim(value=0.8873239436619718, attributes={'year': 0.7746478873239436, 'manufacturer': 1.0}),
 97: AttributeValueSim(value=0.880281690140845, attributes={'year': 0.7605633802816901, 'manufacturer': 1.0}),
 249: AttributeValueSim(value=0.880281690140845, attributes={'year': 0.7605633802816901, 'manufacturer': 1.0}),
 188: AttributeValueSim(value=0.8732394366197183, attributes={'year': 0.7464788732394366, 'manufacturer': 1.0}),
 474: AttributeValueSim(value=0.8732394366197183, attributes={'year': 0.7464788732394366, 'manufactu